The sample dataset [apache](https://github.com/gdv/foundationsCS/tree/master/students/ex-data/apache) contains the files *access.log* and *error.log* that contains the logfile of the accesses to a web server and the errors.
The *access.log* is in [Common Log Format](https://en.wikipedia.org/wiki/Common_Log_Format).
The entries in *error.log* usually have a corresponding entry in *access.log*

1.  Read the file *access.log*
1.  Count the number of accesses (number of lines) made by an IP number
1.  Count the number of successful accesses (status 200) made by an IP number
1.  Count the number of accesses for each directory served
1.  For each origin, count the number of successful accesses
1.  For each origin, count the number of unsuccessful accesses, split according to the
    status code
1.  From the results of the previous point, add a column with the error class (the first
    digit of the status code)
1.  Cluster the accesses in 5-minutes time slices (e.g. from 14:00 to 14:05, from 14:05 to
    14:10, etc). Count the number of accesses for each time slice
1.  Count the number of accesses between each pair of `[info]` or `[error]` entries of *error.log*

### Extra points

1.  For `[info]` entry of *error.log*, find the next entry of *access.log*. For
    example, when considering the entry at `Sun Mar  7 18:00:09 2004`, we want to find the
    entry at `[07/Mar/2004:18:02:10 -0800]`
1.  Count the number of times that the two accesses of the previous point have the same origin.


## Read the file *access.log*

In [30]:
!ls

access.log
error.log
py-09-Apache.ipynb


In [2]:
import pandas as pd

Since the first row of the file *access.log* does not contain the names of the columns, we use the `names` option. Moreover, we use a custom separator, otherwise the fields `type`, `url`, and `prot` would be combined together.

In [36]:
access_log = pd.read_csv('access.log', sep='[\s\t]+', engine='python', header=None, names=['origin',
                                                                                          'identity',
                                                                                          'user',
                                                                                          'time',
                                                                                          'tz',
                                                                                          'type',
                                                                                          'url',
                                                                                          'prot',
                                                                                          'status',
                                                                                          'size'])
access_log.head()

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,[07/Mar/2004:16:05:49,-0800],"""GET",/twiki/bin/edit/Main/Double_bounce_sender?topi...,"HTTP/1.1""",401.0,12846
1,64.242.88.10,-,-,[07/Mar/2004:16:06:51,-0800],"""GET",/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,"HTTP/1.1""",200.0,4523
2,64.242.88.10,-,-,[07/Mar/2004:16:10:02,-0800],"""GET",/mailman/listinfo/hsdivision,"HTTP/1.1""",200.0,6291
3,64.242.88.10,-,-,[07/Mar/2004:16:11:58,-0800],"""GET",/twiki/bin/view/TWiki/WikiSyntax,"HTTP/1.1""",200.0,7352
4,64.242.88.10,-,-,[07/Mar/2004:16:20:55,-0800],"""GET",/twiki/bin/view/Main/DCCAndPostFix,"HTTP/1.1""",200.0,5253


In [43]:
access_log['time'] = access_log['time'].str.replace('[', "" , regex=True)
access_log.head(3)

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,07/Mar/2004:16:05:49,-0800],"""GET",/twiki/bin/edit/Main/Double_bounce_sender?topi...,"HTTP/1.1""",401.0,12846
1,64.242.88.10,-,-,07/Mar/2004:16:06:51,-0800],"""GET",/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,"HTTP/1.1""",200.0,4523
2,64.242.88.10,-,-,07/Mar/2004:16:10:02,-0800],"""GET",/mailman/listinfo/hsdivision,"HTTP/1.1""",200.0,6291


In [44]:
access_log['tz'] = access_log['tz'].str.replace(']', "" , regex=True)
access_log.head(3)

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,07/Mar/2004:16:05:49,-0800,"""GET",/twiki/bin/edit/Main/Double_bounce_sender?topi...,"HTTP/1.1""",401.0,12846
1,64.242.88.10,-,-,07/Mar/2004:16:06:51,-0800,"""GET",/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,"HTTP/1.1""",200.0,4523
2,64.242.88.10,-,-,07/Mar/2004:16:10:02,-0800,"""GET",/mailman/listinfo/hsdivision,"HTTP/1.1""",200.0,6291


In [46]:
access_log['type'] = access_log['type'].str.replace('"', "" , regex=True)
access_log.head(3)

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,07/Mar/2004:16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,"HTTP/1.1""",401.0,12846
1,64.242.88.10,-,-,07/Mar/2004:16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,"HTTP/1.1""",200.0,4523
2,64.242.88.10,-,-,07/Mar/2004:16:10:02,-0800,GET,/mailman/listinfo/hsdivision,"HTTP/1.1""",200.0,6291


In [47]:
access_log['prot'] = access_log['prot'].str.replace('"', "" , regex=True)
access_log.head(3)

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,07/Mar/2004:16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846
1,64.242.88.10,-,-,07/Mar/2004:16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523
2,64.242.88.10,-,-,07/Mar/2004:16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291


In [52]:
access_log['time'] = pd.to_datetime(access_log['time'], format="%d/%b/%Y:%H:%M:%S")
access_log['time'].head()

0   2004-03-07 16:05:49
1   2004-03-07 16:06:51
2   2004-03-07 16:10:02
3   2004-03-07 16:11:58
4   2004-03-07 16:20:55
Name: time, dtype: datetime64[ns]

## Count the number of accesses (number of lines) made by an IP number

We use fancy indexing (maschera booleana) to filter from `access` only the rows where `origin` consists of an IP address. While an IP address consists of 4 numbers in the interval `[0,255]` separated by dots, a simpler regex suffices.

In [55]:
import re

In [67]:
access_log[access_log['origin'].str.contains('\d+\.\d+\.\d+\.\d+')]['origin'].count()

861

If I really want a tighter regex, I can force the fact that numbers have at most three digits.

In [85]:
ip_preciso = access_log[access_log['origin'].str.contains('^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$')]
ip_preciso

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291
3,64.242.88.10,-,-,2004-03-07 16:11:58,-0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,7352
4,64.242.88.10,-,-,2004-03-07 16:20:55,-0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,5253
...,...,...,...,...,...,...,...,...,...,...
1539,10.0.0.153,-,-,2004-03-12 12:23:41,-0800,GET,/dccstats/stats-hashes.1month.png,HTTP/1.1,200.0,1636
1540,10.0.0.153,-,-,2004-03-12 12:23:41,-0800,GET,/dccstats/stats-spam.1year.png,HTTP/1.1,200.0,2262
1541,10.0.0.153,-,-,2004-03-12 12:23:41,-0800,GET,/dccstats/stats-spam-ratio.1year.png,HTTP/1.1,200.0,1906
1542,10.0.0.153,-,-,2004-03-12 12:23:41,-0800,GET,/dccstats/stats-hashes.1year.png,HTTP/1.1,200.0,1582


Then we can group the rows with the same origin and count the size of each group

In [88]:
ip_preciso.groupby('origin').size()

origin
10.0.0.153         270
12.22.207.235        1
128.227.88.79       14
142.27.64.35         7
145.253.208.9        7
194.151.73.43        4
195.11.231.210       1
195.230.181.122      1
195.246.13.119      12
200.222.33.33        1
203.147.138.233     13
207.195.59.160      20
208.247.148.12       4
212.21.228.26        1
212.92.37.62        14
213.181.81.4         1
216.139.185.45       1
219.95.17.51         1
4.37.97.186          1
61.165.64.6          4
61.9.4.61            3
64.242.88.10       452
64.246.94.141        1
64.246.94.152        1
66.213.206.2         1
67.131.107.5         3
dtype: int64

## Count the number of successful accesses (status 200) made by an IP number

We only have to filter the rows with status equal to 200

In [101]:
access_log[access_log['status'] == 200.0]['status'].count()

1274

An alternative version uses the `len` function.

In [102]:
len(access_log[access_log['status'] == 200.0])

1274

In [124]:
access_log['url'].head(3)

0    /twiki/bin/edit/Main/Double_bounce_sender?topi...
1    /twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....
2                         /mailman/listinfo/hsdivision
Name: url, dtype: object

## Count the number of accesses for each directory served

First we add a column `dir` to each row

The first step is to build a function, called `extract_dir`, that computes the directory from a url.

In [134]:
# Se volessi usare str.extract, il concetto è diverso rispetto a contains:
# dove contains controlla se nella stringa di riferimento il pattern esista o meno
# ritornando, ad esempio, una maschera booleana, con extract si estrae il pattern
# a cui viene assegnato il gruppo/stringa nel quale viene ritrovato:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html

access_log[access_log['url'].str.match('^.*\/')].head(3)

,origin,identity,user,time,tz,type,url,prot,status,size
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291


In [143]:
access_log[~ access_log['url'].str.match('^.*\/')]

,origin,identity,user,time,tz,type,url,prot,status,size
95,80-219-148-207.dclient.hispeed.ch,-,-,2004-03-07 19:47:36,-0800,OPTIONS,*,HTTP/1.0,200.0,-
906,h194n2fls308o1033.telia.com,-,-,2004-03-09 13:49:05,-0800,-,408,-,NaN,None


In [ ]:
# Poiché ci sono alcuni indici che non rientrano nella regex devo eliminarli, altrimenti mi darà errore
# nel momento in cui applicherò la funzione successiva
access_log.drop([95, 906], inplace=True)

In [146]:
def extract_dir(s):
    return re.match('^.*\/', s).group()

In [147]:
access_log['dir'] = access_log.apply(lambda pincopallo: extract_dir(pincopallo['url']), axis=1)
access_log.head(3)

,origin,identity,user,time,tz,type,url,prot,status,size,dir
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291,/mailman/listinfo/


In [150]:
access_log.groupby('dir')['dir'].count()

dir
/                                     184
/MSOffice/                              1
/_vti_bin/                              1
/cgi-bin/                              16
/cgi-bin/mailgraph.cgi/               128
                                     ... 
/twiki/bin/view/Sandbox/                9
/twiki/bin/view/TWiki/                 99
/twiki/pub/TWiki/TWikiDocGraphics/      6
/twiki/pub/TWiki/TWikiLogos/           66
/twiki/view/Main/                       1
Name: dir, Length: 61, dtype: int64

Since a regex can be a brittle solution, we have to check that it is actually correct. More precisely, we are going to check when the regex is not found.

Those two rows are problematic. Moreover, we cannot make any sense of them, so we decide to drop them.

Then we can use `apply`

Since using the `axis` option of `apply` can be confusing, an alternative solution is to build a list correponding to the new column

## For each origin, count the number of successful accesses

In [160]:
access_log[access_log['status'] ==  200.0].groupby('origin').size()

origin
0x503e4fce.virnxx2.adsl-dhcp.tele.dk      2
1-320.cnc.bc.ca                           4
1-729.cnc.bc.ca                           6
10.0.0.153                              187
12.22.207.235                             1
                                       ... 
watchguard.cgmatane.qc.ca                 2
wc03.mtnk.rnc.net.cable.rogers.com        1
wc09.mtnk.rnc.net.cable.rogers.com        3
wwwcache.lanl.gov                         1
yongsan-cache.korea.army.mil              4
Length: 167, dtype: int64

## For each origin, count the number of unsuccessful accesses, split according to the status code

The `groupby` can receive a list of column names

In [166]:
access_log[~ (access_log['status'] ==  200.0)].groupby(['origin', 'status']).size()

origin                                 status
0x503e4fce.virnxx2.adsl-dhcp.tele.dk   304.0       1
1-729.cnc.bc.ca                        302.0       1
10.0.0.153                             302.0       1
                                       304.0      82
128.227.88.79                          304.0       2
142.27.64.35                           302.0       1
                                       304.0       4
145.253.208.9                          304.0       1
1513.cps.virtua.com.br                 404.0       1
195.246.13.119                         401.0       1
2-110.cnc.bc.ca                        304.0       3
207.195.59.160                         304.0       5
                                       401.0       1
61.9.4.61                              404.0       2
64.242.88.10                           401.0     112
68-174-110-154.nyc.rr.com              304.0       1
92-moc-6.acn.waw.pl                    304.0       1
cpe-203-51-137-224.vic.bigpond.net.au  302.0       1


## From the results of the previous point, add a column with the error class (the first digit of the status code)

In [172]:
gruppi = access_log[~ (access_log['status'] ==  200.0)].groupby(['origin', 'status'], as_index=False).size()
gruppi.head(2)

,origin,status,size
0,0x503e4fce.virnxx2.adsl-dhcp.tele.dk,304.0,1
1,1-729.cnc.bc.ca,302.0,1


Since the `status` field is part of the index, we have to move it to a column name, via `reset_index`

In [178]:
gruppi['classe'] = gruppi['status'].astype(int) // 100

Now we can add the desired column

In [179]:
gruppi.head()

,origin,status,size,classe
0,0x503e4fce.virnxx2.adsl-dhcp.tele.dk,304.0,1,3
1,1-729.cnc.bc.ca,302.0,1,3
2,10.0.0.153,302.0,1,3
3,10.0.0.153,304.0,82,3
4,128.227.88.79,304.0,2,3


## Cluster the accesses in 5-minutes time slices (e.g. from 14:00 to 14:05, from 14:05 to 14:10, etc). Count the number of accesses for each time slice

We use a procedure similar to the previous point. Notice that we need only the hour and the minute (not the full timestamp) to build the clusters.

In [175]:
access_log.head(2)

,origin,identity,user,time,tz,type,url,prot,status,size,dir
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/


In [192]:
access_log['cluster'] = (access_log['time'].dt.hour * 12) + (access_log['time'].dt.minute // 5)
access_log

,origin,identity,user,time,tz,type,url,prot,status,size,dir,cluster
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291,/mailman/listinfo/,194
3,64.242.88.10,-,-,2004-03-07 16:11:58,-0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,7352,/twiki/bin/view/TWiki/,194
4,64.242.88.10,-,-,2004-03-07 16:20:55,-0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,5253,/twiki/bin/view/Main/,196
...,...,...,...,...,...,...,...,...,...,...,...,...
1541,10.0.0.153,-,-,2004-03-12 12:23:41,-0800,GET,/dccstats/stats-spam-ratio.1year.png,HTTP/1.1,200.0,1906,/dccstats/,148
1542,10.0.0.153,-,-,2004-03-12 12:23:41,-0800,GET,/dccstats/stats-hashes.1year.png,HTTP/1.1,200.0,1582,/dccstats/,148
1543,216.139.185.45,-,-,2004-03-12 13:04:01,-0800,GET,/mailman/listinfo/webber,HTTP/1.1,200.0,6051,/mailman/listinfo/,156
1544,pd95f99f2.dip.t-dialin.net,-,-,2004-03-12 13:18:57,-0800,GET,/razor.html,HTTP/1.1,200.0,2869,/,159


In [193]:
access_log.groupby('cluster').size()

cluster
1      6
2      1
3      3
4      3
5      7
      ..
283    5
284    2
285    1
286    2
287    3
Length: 267, dtype: int64

## For `[info]` entry of *error.log*, find the next entry of *access.log*. 

*For example, when considering the entry at `Sun Mar  7 18:00:09 2004`, we want to find the entry at `[07/Mar/2004:18:02:10 -0800]`*

Each error has a corresponding (i.e. same date, time, origin) entry in *access.log*

In [199]:
error_log = pd.read_csv('error.log', header=None)
error_log.head()

,0
0,[Sun Mar 7 16:02:00 2004] [notice] Apache/1.3...
1,[Sun Mar 7 16:02:00 2004] [info] Server built...
2,[Sun Mar 7 16:02:00 2004] [notice] Accept mut...
3,[Sun Mar 7 16:05:49 2004] [info] [client 64.2...
4,[Sun Mar 7 16:45:56 2004] [info] [client 64.2...


The first step is to extract the field corresponding to the date/time.

In [206]:
# con il punto interrogativo gli dico di trovare quanto scritto prima
# o 0 o 1 volta: dato che, dopo la prima parentesi quadra, la seconda è quella
# che chiude riesco in questo modo ad estrarre solamente il campo della data e
# dell'ora.
error_log['dateTime'] = error_log[0].str.extract('^\[(.*?)]')
error_log.head()

,0,dateTime
0,[Sun Mar 7 16:02:00 2004] [notice] Apache/1.3...,Sun Mar 7 16:02:00 2004
1,[Sun Mar 7 16:02:00 2004] [info] Server built...,Sun Mar 7 16:02:00 2004
2,[Sun Mar 7 16:02:00 2004] [notice] Accept mut...,Sun Mar 7 16:02:00 2004
3,[Sun Mar 7 16:05:49 2004] [info] [client 64.2...,Sun Mar 7 16:05:49 2004
4,[Sun Mar 7 16:45:56 2004] [info] [client 64.2...,Sun Mar 7 16:45:56 2004


In [ ]:
error_log['dateTime'] = pd.to_datetime(error_log['dateTime'])
error_log['dateTime']

Then we extract the type of the error

In [233]:
error_log['type_error'] = error_log[0].str.extract('^\[.*?\]\s\[(.*?)\]')
error_log

,0,dateTime,type_error
0,[Sun Mar 7 16:02:00 2004] [notice] Apache/1.3...,2004-03-07 16:02:00,notice
1,[Sun Mar 7 16:02:00 2004] [info] Server built...,2004-03-07 16:02:00,info
2,[Sun Mar 7 16:02:00 2004] [notice] Accept mut...,2004-03-07 16:02:00,notice
3,[Sun Mar 7 16:05:49 2004] [info] [client 64.2...,2004-03-07 16:05:49,info
4,[Sun Mar 7 16:45:56 2004] [info] [client 64.2...,2004-03-07 16:45:56,info
...,...,...,...
103,[Mon Mar 8 14:54:56 2004] [info] [client 64.2...,2004-03-08 14:54:56,info
104,[Tue Mar 9 13:49:05 2004] [info] [client 81.2...,2004-03-09 13:49:05,info
105,[Wed Mar 10 11:45:51 2004] [info] [client 24.7...,2004-03-10 11:45:51,info
106,[Thu Mar 11 02:27:34 2004] [error] [client 200...,2004-03-11 02:27:34,error


Then we parse the date/time

We add a field `next` which is the index of the next row. We exploit the fact that, once we reset the index, the index is a sequence of consecutive integers starting from zero and that we can build a column from a list of values.
To the purpose, the `access` dataframe has to be sorted by increasing `datetime`.

In [248]:
access_log.sort_values(by='time', inplace=True)
access_log.head()

,origin,identity,user,time,tz,type,url,prot,status,size,dir,cluster
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291,/mailman/listinfo/,194
3,64.242.88.10,-,-,2004-03-07 16:11:58,-0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,7352,/twiki/bin/view/TWiki/,194
4,64.242.88.10,-,-,2004-03-07 16:20:55,-0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,5253,/twiki/bin/view/Main/,196


Since each error has a corresponding entry in the `access.log` file, we merge the two dataframes.

In [249]:
access_log['next'] = list(range(1, len(access_log) + 1))
access_log.head()

,origin,identity,user,time,tz,type,url,prot,status,size,dir,cluster,next
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193,1
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193,2
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291,/mailman/listinfo/,194,3
3,64.242.88.10,-,-,2004-03-07 16:11:58,-0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,7352,/twiki/bin/view/TWiki/,194,4
4,64.242.88.10,-,-,2004-03-07 16:20:55,-0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,5253,/twiki/bin/view/Main/,196,5


In [250]:
error_log.head(3)

,0,dateTime,type_error
0,[Sun Mar 7 16:02:00 2004] [notice] Apache/1.3...,2004-03-07 16:02:00,notice
1,[Sun Mar 7 16:02:00 2004] [info] Server built...,2004-03-07 16:02:00,info
2,[Sun Mar 7 16:02:00 2004] [notice] Accept mut...,2004-03-07 16:02:00,notice


In [256]:
fuso = pd.merge(access_log, error_log, left_on='time', right_on='dateTime', how='outer')
fuso.head()

,origin,identity,user,time,tz,type,url,prot,status,size,dir,cluster,next,0,dateTime,type_error
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193.0,1.0,[Sun Mar 7 16:05:49 2004] [info] [client 64.2...,2004-03-07 16:05:49,info
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193.0,2.0,NaN,NaT,NaN
2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291,/mailman/listinfo/,194.0,3.0,NaN,NaT,NaN
3,64.242.88.10,-,-,2004-03-07 16:11:58,-0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,7352,/twiki/bin/view/TWiki/,194.0,4.0,NaN,NaT,NaN
4,64.242.88.10,-,-,2004-03-07 16:20:55,-0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,5253,/twiki/bin/view/Main/,196.0,5.0,NaN,NaT,NaN


Check if the rows of `error` are in `merged`. The following query cannot return any row.

Finally, use the `next` field to merge `merged` and `found`.

In [257]:
totale = pd.merge(fuso, fuso, left_index=True, right_on='next')
totale.head()

,next,origin_x,identity_x,user_x,time_x,tz_x,type_x,url_x,prot_x,status_x,...,url_y,prot_y,status_y,size_y,dir_y,cluster_y,next_y,0_y,dateTime_y,type_error_y
0,1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,...,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193.0,1.0,[Sun Mar 7 16:05:49 2004] [info] [client 64.2...,2004-03-07 16:05:49,info
1,2,64.242.88.10,-,-,2004-03-07 16:10:02,-0800,GET,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,...,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193.0,2.0,NaN,NaT,NaN
2,3,64.242.88.10,-,-,2004-03-07 16:11:58,-0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,...,/mailman/listinfo/hsdivision,HTTP/1.1,200.0,6291,/mailman/listinfo/,194.0,3.0,NaN,NaT,NaN
3,4,64.242.88.10,-,-,2004-03-07 16:20:55,-0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,...,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200.0,7352,/twiki/bin/view/TWiki/,194.0,4.0,NaN,NaT,NaN
4,5,64.242.88.10,-,-,2004-03-07 16:23:12,-0800,GET,/twiki/bin/oops/TWiki/AppendixFileSystem?templ...,HTTP/1.1,200.0,...,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200.0,5253,/twiki/bin/view/Main/,196.0,5.0,NaN,NaT,NaN


## Count the number of times that the two accesses of the previous point have the same origin.

In [260]:
len(totale[totale['origin_x'] == totale['origin_y']])

1219

## Count the number of accesses between each pair of `[info]` or `[error]` entries of *error.log*

In [261]:
access_log.head(2)

,origin,identity,user,time,tz,type,url,prot,status,size,dir,cluster,next
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193,1
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193,2


In [262]:
error_log.head(2)

,0,dateTime,type_error
0,[Sun Mar 7 16:02:00 2004] [notice] Apache/1.3...,2004-03-07 16:02:00,notice
1,[Sun Mar 7 16:02:00 2004] [info] Server built...,2004-03-07 16:02:00,info


In [264]:
accesso_con_errori = pd.merge(access_log, error_log, left_on='time', right_on='dateTime', how = 'outer')
accesso_con_errori.head(2)

,origin,identity,user,time,tz,type,url,prot,status,size,dir,cluster,next,0,dateTime,type_error
0,64.242.88.10,-,-,2004-03-07 16:05:49,-0800,GET,/twiki/bin/edit/Main/Double_bounce_sender?topi...,HTTP/1.1,401.0,12846,/twiki/bin/edit/Main/,193.0,1.0,[Sun Mar 7 16:05:49 2004] [info] [client 64.2...,2004-03-07 16:05:49,info
1,64.242.88.10,-,-,2004-03-07 16:06:51,-0800,GET,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1....,HTTP/1.1,200.0,4523,/twiki/bin/rdiff/TWiki/,193.0,2.0,NaN,NaT,NaN


We are going to exploit the fact that we have a column `index` of `merged` that contains the position inside the `access` dataframe. So we have to compute the difference of the index between two consecutive entries that are errors or info.

Let us start by isolating such entries.

In [268]:
accesso_con_errori[(accesso_con_errori['type_error'] == 'info') | (accesso_con_errori['type_error'] == 'error')]['next']

0          1.0
17        18.0
30        31.0
35        36.0
39        40.0
         ...  
1081    1082.0
1545       NaN
1549       NaN
1550       NaN
1551       NaN
Name: next, Length: 106, dtype: float64

Methods on dataframe are mostly designed to process each row independently from each other. Hence we prefer to transform the series into a list.

In [ ]:
lista = list(accesso_con_errori[(accesso_con_errori['type_error'] == 'info') | (accesso_con_errori['type_error'] == 'error')]['next'])
lista

In [291]:
del lista[-1]

In [301]:
len(lista) + 1

103

In [312]:
l_2 = []
for elem in range(1, len(lista)):
    l_2.append(lista[elem] - lista[elem - 1] - 1)

l_2

In [ ]:
l_2 = []
primi = lista[:-1]
secondi = lista[1:]

for elem in range(1, len(primi)):
    l_2.append(secondi[elem] - primi[elem] - 1)

l_2

Now we scan the list, except for the first element, and we compute the difference between the current and the previous element.

This approach requires managing the index of the list.

An easier way is to extract two sublists of `info_errors_list`: the first removing the first element, and the second removing the last element. Those two sublists have the same length and are coordinated, that is in position `i` there are two elements that are related (actually, they are the two operands of the difference).

An even easier way is to exploit the fact that the two sublists are coordinated. This allows to use a zip to couple each pair of related elements, and a list comprehension to obtained the desired difference.

In [ ]:
[b - a - 1 for (a,b) in list(zip(lista[:-1], lista[1:]))]